## This Notebook - Goals - FOR EDINA

**What?:**
- K-means compression of image data, additional image filters
- Interactive

**Who?:**
- Intro to ML course (3rd year computer science UG)
- PG students in social sciences learning ML
- Design course
- Programming course (functions, for loops, types and overflow)

**Why?:**
- Visual
- Wide audience
- Show progression of steps and concepts

**Noteable features to exploit:**
- Visualisation
- Interactive (widgets!)
- Tutorial format

**How? Tools/methods used:**
- Widgets

<hr>

# Create your own selfie filters!

This notebook will guide you through the process of changing an image using python.

<hr>

## Context

### Python

If you're learning python, [this page](https://learnxinyminutes.com/docs/python/) provides a great quick reference.

Having said that, **you do not need to know python to follow along with this tutorial**.

### RGB

The functions below manipulate an array of **numbers** in order to change how an image **loooks** looks. How exactly does this work?

The array that is manipulated in all of the following functions is a $(M,N,3)$ array of integers, where $M$ is the height of the image in pixels and $N$ is the width of the image, in pixels.

The depth of this array can be thought of like 3 layers, where each layer contains information about a specific colour: red, green or blue [(RGB)](https://www.rapidtables.com/web/color/RGB_Color.html). 

So the first layer, which deals with red, is a $(M,N)$ array that contains a number for every pixel in the image, to represent the amount of red in that pixel. The value is between 0 and 255.

If all RGB values are 0, the pixel is black. If all RGB values are 255, the pixel is white.

For example: suppose the $(100,50,1)th$ element of the array contains the value 0. This means that the pixel at row 100 and column 50 has no green at all. 

<hr>

# Getting started - set up

## Import useful modules

Python has a [huge](https://scikit-image.org) [amount](https://docs.opencv.org/master/dc/dd3/tutorial_gausian_median_blur_bilateral_filter.html) [of](https://imagemagick.org/Usage/photos/#color) [support](http://www.spectralpython.net/graphics.html) for image processing, so this notebook only covers a couple of possibilities. 

All modules linked are pre-installed on Noteable, so they just need to be imported!

In [ ]:
# displaying images/data
import matplotlib.pyplot as plt
# reading images to arrays
from matplotlib.image import imread
# magic iPython command for visualisations in notebooks
%matplotlib inline

# implementation of K-means algorithm
from sklearn.cluster import KMeans

# array functions
import numpy as np

# used for interactive slider bars at end
import ipywidgets as widgets

## Loading and displaying images

<i><b>Note:</b> The last section of this notebook will help you to upload your own image.</i>

Use the [imread()](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imread.html) function to load an image into a [numpy array](https://scipy-lectures.org/intro/numpy/array_object.html). 

<i><b>Challenge:</b> Can you see why 'int16' type is neccessary?<br><b>Hint:</b> Consider the case where a pixel is altered to a negative number but is stored in an unsigned integer type.</i>

<div class="alert alert-info">
    <b>What do these cells do?</b><br>
    1. The first cell defines a function, <code>array_img(imgFile)</code> that reads an image and specifies the type that elements should be stored as.<br>
    2. The second cell loads a particular image using the function that was just defined.<br>
</div>

In [ ]:
# function that loads an image and converts to array using imread()
def array_img(imgFile):    
    # load image to 3D numpy array
    img = imread(imgFile).astype('int16')
    return img

In [ ]:
# load picture into array
img_arr = array_img('JamesFace.jpg')

<div class="alert alert-info">
    <b>What do these cells do?</b><br>
    1. The first cell defines a function, <code>show_img(*imgs)</code> that displays an image or images.<br>
    2. The second cell uses this function to display the <i>"woman_photo.jpg"</i> picture.
</div>

In [ ]:
# function to display one or two images
def show_img(*imgs):
    # if there are 2 image arguments
    if (len(imgs)==2):
        # plot both images side by side
        fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,20))
        ax1.imshow(imgs[0])
        ax2.imshow(imgs[1])
        ax1.axis('off')
        ax2.axis('off')
        # give title for comparison
        ax1.set_title("Before")
        ax2.set_title("After")
    # if there is only one image argument
    elif (len(imgs)==1):
        # plot the image
        plt.figure(figsize=(7,7))
        plt.axis('off')
        _ = plt.imshow(imgs[0])

In [ ]:
# display picture of woman
show_img(img_arr)

<hr>

# Compress an Image with K-means

[K-means](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_iris.html) is an algorithm that helps to group data.

Groups *("clusters")* are each made up of points that are similar to each other. 

In this example, the algorithm will try to:
- Find colours that are similar to each other and group them.
- Find the average colour for each group, which becomes the new palette.
- Map the pixels in the original image down to only the colours specified by this reduced palette.

Since the resulting image can use group lookup to represent colours, a smaller data type can be used to handle each pixel. Storage benefits aside, it produces interesting effects on images.

### Running K-means

Scikit-learn has it's own [implementation of K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), which is imported in the code cell at the top of this notebook. 

Run the K-means algorithm by creating the model, then fitting it on our image. The groupings created by K-means can then be used to reduce the number of colours used in the image.

### Reshape step(s)

<b>Reshape at beginning:</b> For a colour image that is $M$ pixels tall and $N$ pixels wide, <code>imread()</code> produces a <code>(M,N,3)</code> numpy array. 

To use the algorithm, the matrix needs to be reshaped to a 2-dimensional <code>(M*N,3)</code> array where each row represents a pixel, and contains the RGB information in its columns.

<i><b>Note:</b> This hasnt changed the data at all, just the order in which the data is arranged.</i>

<b>Reshape at end:</b>The output of the compress function will be the same shape as the input, <code>(M,N,3)</code> array. Therefore, the array will also be reshaped at the end.

<div class="alert alert-info">
<b>What do these cells do?:</b><br>
    1. The first cell defines a function,<code>compress_img(img, n)</code> that puts the steps above together by reshaping the array, running K-means, then reshaping again.<br>
    2. The second cell applies that function to our specific image, then uses the <code>show_img(*imgs)</code> function to display it.
</div>

In [ ]:
# function to reshape image and map down to n colours
# default n = 10
def compress_img(img, n=10):
    # store original size to convert back to later
    size = img.shape
    # reshape image array to 2-dimensions
    r_img = img.reshape(size[0]*size[1], size[2])
    
    # set limits on allowed values for n
    if (n>25 or n<1):
        raise Exception("n must be in range 0<n<26.")
    
    # create a KMeans model with n clusters
    km_model = KMeans(n_clusters=n)
    # fit model to image colour data
    km_model.fit(r_img)
    # retrieve average colour for each group
    # identify appropriate colour by label
    c_img = km_model.cluster_centers_[km_model.labels_]
    # restrict values to range allowed by RGB
    c_img = np.clip(c_img.astype(int), 0, 255)
    
    # reshape new image data to original shape
    c_img = c_img.reshape(size[0], size[1], size[2])

    return c_img

In [ ]:
# apply function to woman image, 15 colours
mypic_compressed = compress_img(img_arr, 15)
show_img(img_arr,mypic_compressed)

<hr>

# Incorporating more filters

So far, the following functions have been defined:

- <code>array_img(imgFile)</code>: takes image file and converts to 3-dimensional numpy array.
- <code>show_img(*imgs)</code>: display image(s) from 3-dimensional array.
- <code>compress_img(img,n=10)</code>: given a 3-dimensional array representing an image, reduce the number of colours to n, where n is 10 by default.

The next part of this tutorial will show how to define the following functions:

- <code>lighter_img(img, n=20)</code>: increase/decrease lightness by n. Default n=20.
- <code>warmer_img(img, n=20)</code>: increase/decrease warmth by n. Default n=20.
- <code>contrast_img(img, scl=1.25)</code>: increase/decrease contrast by scl. Default scl=1.25

<hr>

## Make image lighter or darker

Because of the way RGB works, images can be made lighter by simply increasing the value of each of each colour by an equal amount per pixel. Similarly, the image can be made darker by reducing the RGB values for each pixel.



In [ ]:
# function to make image lighter or darker
# default: increase lightness by 20
def lighter_img(img, n=20):
    # increment values of each pixel by n
    img = img + n
    # restrict values to range allowed by RGB
    l_img = np.clip(img.astype(int), 0, 255)
    return l_img

The function can be given a **positive number** to make the image **lighter**, or a **negative number** to make the image **darker**.

Take a look at this in action:

In [ ]:
# make image lighter by the default value, 20
mypic_lighter = lighter_img(img_arr)
show_img(img_arr, mypic_lighter)

In [ ]:
# make image darker by using a negative n value
mypic_darker = lighter_img(img_arr, -75)
show_img(img_arr, mypic_darker)

## Make image warmer/cooler

Once you've understood the lighter/darker idea, it's easy to extend this to change the colour in a custom way.

As an example, here is a warmth function that increases the red by n and the blue by a third of n in order to make the image warmer (or cooler if n is negative).

<i><b>Note:</b> It might seem strange to use red and blue to change the amount of orange in the picture. This is because RGB colours are additive. For additive colours, red + blue = yellow.</i>

In [ ]:
# function to alter warmth of image
# default: increase warmth by 20
def warmer_img(img, n=20):
    # change r and b values to make image more or less orange
    r_img = img[:,:,0] + n
    b_img = img[:,:,1] + (n//3)
    g_img = img[:,:,2]
    # stack arrays depthwise to get back to (M,N,3) shaped array
    w_img = np.dstack((r_img, b_img, g_img))
    # restrict values to range allowed by RGB
    d_img = np.clip(w_img.astype(int), 0, 255)
    return d_img

Take a look at this in action with our picture:

In [ ]:
# make image warmer by 30
w_img = warmer_img(img_arr,30)
show_img(img_arr,w_img)

In [ ]:
### make image cooler by 20
c_img = warmer_img(img_arr,-20)
show_img(img_arr, c_img)

## Change contrast in an image

This is a little more complicated than the previous two filters, but uses a very similar idea.

Going from the basics: to increase the contrast, the lighter bits of the image need to become even lighter, and the darker bits need to get even darker.

From the <code>lighter_img()</code> function, its already clear that adding some value to the RGB values of each pixel makes the whole image lighter. That's a great starting point:

Therefore, the **"light" pixels need to be given higher RGB values, and the "dark" pixels need to be given lower RGB values**.

To categorise a pixel as light or dark, there needs to be some **threshold/neutral value**. If the RGB value of a pixel is **above** the threshold, the pixel is considered "light". If it's **below** the threshold, its "dark".

### Where is the threshold?

**Plan A:** One approach could be to take the midpoint of all possible colour values to be the threshold. The range of possible values is 0 to 255, so the midpoint is 127 (rounded down). If we use this on an image with a wide range of values, the result might be ok. But take a look at what happens when this method is used on a dark picture: 

In [ ]:
# copy the dark image (above)
too_dark_img = mypic_darker.copy()

# take the midpoint of 0 and 255 to be the neutral point:
mid = (255 - 0)//2

# add 20 to pixels lighter than mid
too_dark_img[too_dark_img>mid] +=50
too_dark_img[too_dark_img<mid] -=50

# restrict values to range allowed by RGB
dark_img = np.clip(too_dark_img, 0, 255)

# display
show_img(mypic_darker, dark_img)

**What went wrong?**:

Since most of the pixels are darker than the midpoint, the picture ends up darker with a few light patches! 

**Plan B:** So the threshold needs to be the midpoint of the pixels' lightness/darkess. That is, the mid-point for that *specific* picture. This can be found by taking the mean average of all RGB values in the whole picture.

In [ ]:
# copy the dark image (above)
too_dark_img_2 = mypic_darker.copy()

# take the average RGB value to be the neutral point
mid = np.mean(too_dark_img_2)

# add 50 to pixels lighter than mid
too_dark_img_2[too_dark_img_2>mid] +=50
# subtract 50 from pixels darker than mid
too_dark_img_2[too_dark_img_2<mid] -=50

# restrict values to range allowed by RGB
less_dark_img = np.clip(too_dark_img_2, 0, 255)

# display
show_img(mypic_darker,less_dark_img)

That's a bit better!

**Choose plan B (for now)**

With the threshold problem solved, there is something else that can improve the result.

### Using a multiplier

**Multiplier:** Instead of adding or subtracting some number to the RGB values, they can be multiplied by a scalar.

To make this work, the values for the image all need to be shifted down so that the threshold can be treated as a neutral point. This way, the multiplier pushes values towards/away from the neutral point, without needing to fiddle with positive and negative multipliers. Additionally, reducing the contrast becomes much easier, because a multiplier that is less than 1 will pull colour values towards the average.

Let's see it this action:

In [ ]:
# copy the dark image (above)
too_dark_img_3 = mypic_darker.copy()

# take the average RGB value to be the neutral point
avg = np.mean(too_dark_img_3)

# shift the whole array down (centralise)
c_img = too_dark_img_3 - avg
# multiply all values by the contrast multiplier of 2 (away from origin)
c_img = c_img*2
# shift the values back up
c_img = c_img + avg

# restrict values to range allowed by RGB
d_img = np.clip(c_img.astype(int), 0, 255)

# display
show_img(mypic_darker,d_img)

Extend this idea beyond the average colour. Instead, treat the average of each colour as the origin *for just that colour*.

Consider now contrast where each colour (r, g, b) has its own neutral based on its own average:

In [ ]:
# function to change contrast in image
# default scalar = 1.25 (increase contrast)
def contrast_img(img,scl=1.25):
    
    # for each colour (RGB):
    for k in range(3):
        # find average value for colour
        avg = np.mean(img[:,:,k])
        # centralise data about average (shift down)
        ctd = img[:,:,k] - avg
        # scale by scl parameter value
        scld = ctd*scl
        # shift back up
        updata = scld + avg
        # stack new values depthwise to a 3D array
        if (k==0):
            arr = updata
        else:
            arr = np.dstack((arr, updata))
            
    # restrict values to range allowed by RGB     
    c_img = np.clip(arr.astype(int), 0, 255)
    
    return c_img

In [ ]:
# compare with the alternative contrast method
show_img(d_img, contrast_img(img_arr, 2))

In [ ]:
# scalar less than 1 reduces contrast
show_img(img_arr,contrast_img(img_arr,0.5))

<hr>

# Putting it all together

Now we have the following functions we can use to interact with images:

- <code>array_img(imgFile)</code>: reads the image from file to array
- <code>show_img(*imgs)</code>: display one or two images

Filters:
- <code>compress_img(img, n=10)</code>: compress image using K-means with n colours. Default n=10.
- <code>lighter_img(img, n=20)</code>: increase/decrease lightness by n. Default n=20.
- <code>warmer_img(img, n=20)</code>: increase/decrease warmth by n. Default n=20.
- <code>contrast_img(img, scl=1.25)</code>: increase/decrease contrast by scl. Default scl=1.25

<b>The rest of this notebook</b> shows how to use the filters interactively, and optionally use them on your own image.


<hr>

## Upload your image (optional)

<b>Option 1:</b> If you already have an image in the same directory as this notebook, you can replace 'woman_photo.jpg' with the name of your image in the cell below:

In [ ]:
# set image name
image = array_img('JamesFace.jpg')

<b>Option 2:</b> You can select an image from your computer by clicking the "Upload" button below:

<i><b>Note:</b> If you wish to repeat this step and upload another picture, you need to execute the cell below again, which will reset the upload. Please be aware that your old image will be overridden if you do this.</i>

In [ ]:
# If you wish to try more images after the first one you upload
# you will need to execute this cell which will reset the uploads
pic = widgets.FileUpload(accept='image/*',multiple=False)
pic

Regardless of which option you chose, run all cells below this one:

In [ ]:
### WARNING: DO NOT change anything in this cell.
try:
    upl_file = next(iter(pic.value))
    content = pic.value[upl_file]['content']
    with open('my_pic.jpg','wb') as f: f.write(content)
    image = array_img('my_pic.jpg')
    pic.close()
except:
    print("Did not upload via upload button.")

In [ ]:
# check that the image is as expected
show_img(image)

<hr>

# Interacting with your filters

Multiple filters can be applied to an image by using the output of one filter as the input for the next filter.

<b>Move the slider bars below</b> to change the lightness, warmth, contrast and palette size.

You can safely ignore all the code below (although if you're interested in how it works, follow [this link](https://ipywidgets.readthedocs.io/en/stable/)).

<i><b>Note:</b> if "palette size" is set to 26, the <code>compress_img(img, n)</code> function won't be executed. Additionally, due to the complexity of this function, you will need to be patient if you do change this slider. Try giving it a little wiggle if you've been waiting more than 30 seconds or so.</i>

In [ ]:
# create sliders for each filter
lig = widgets.IntSlider(min=-150,max=150,
                        description='lightness', value=0)
wam = widgets.IntSlider(min=-150,max=150,
                        description='warmth', value=0)
con = widgets.FloatSlider(min=0,max=2,
                        description='contrast', value=1)
c   = widgets.IntSlider(min=1,max=26,
                        description='palette size',value=26)

# set order of implementation
def f(lig, wam, con, c):
    x = lighter_img(image,lig)
    x = warmer_img(x,wam)
    x = contrast_img(x,con)
    # only apply compress if slider is moved
    if (0<c<26):
        x = compress_img(x, c)
    show_img(x)

In [ ]:
# display result of filters on right hand side
out = widgets.interactive_output(f, 
                                 {'lig':lig, 'wam':wam,'con':con,'c':c})
widgets.HBox([widgets.VBox([lig,wam,con,c]), out])